In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))

path,name,size
dbfs:/FileStore/tables/8lines.txt,8lines.txt,95
dbfs:/FileStore/tables/Corona_NLP_test.csv,Corona_NLP_test.csv,1002494
dbfs:/FileStore/tables/Corona_NLP_train.csv,Corona_NLP_train.csv,10500262
dbfs:/FileStore/tables/Finance_news.csv,Finance_news.csv,672006
dbfs:/FileStore/tables/Graph.txt,Graph.txt,87787062
dbfs:/FileStore/tables/Spark_assignment_dag.jpg,Spark_assignment_dag.jpg,4035928
dbfs:/FileStore/tables/allnodeslist.txt,allnodeslist.txt,55741422
dbfs:/FileStore/tables/mnm_dataset-1.csv,mnm_dataset-1.csv,1284872
dbfs:/FileStore/tables/mnm_dataset.csv,mnm_dataset.csv,1284872
dbfs:/FileStore/tables/sentiment140.csv,sentiment140.csv,238803811


In [0]:
targetA = '0'
targetB = '5'
counter = sc.accumulator(0)

In [0]:
# converts line to node
def toNode(line):
    data = line.split()
    target = data[0]
    
    # the nodes which can be reached from target
    connections = data[1:]
    
    # initialize each node with search status as "NOT READY" 
    searchStatus = 'NOT READY'
    
    # initialize distance of each node with a large value
    distance = 10000
    
    # initialize search status of starting node/ targetA as "READY"
    if (target == targetA):
        # as we are starting from the source node or targetA, initialize its distance as 0
        # as its distance from itself is 0.
        searchStatus = 'READY'
        distance = 0
    return (target, (connections, distance, searchStatus))

In [0]:
# apply function on rdd
def transformInput():
    input = sc.textFile("dbfs:/FileStore/tables/8lines.txt")
    # map returns a new rdd by applying the mentioned/passed
    # function to the corresspondind rdd.
    # Here we are applying the toNode() function to each line of input file.
    return input.map(toNode)

In [0]:
def mapNode(node):
    # this node is like something returned by toNode() function.
    target = node[0]
    
    # data has (connections,distance,searchStatus)
    data = node[1] 
    connections = data[0]
    distance = data[1]
    searchStatus = data[2]
    
    results = []
    
    # if this node is ready to search
    if (searchStatus == 'READY'):
        # for all nodes that can be reached from target node
        for connection in connections:
            newTarget = connection
            # set its distance as target node's distance + 1, we will find minimum when we reduce.
            newDistance = distance + 1
            newStatus = 'READY'
            if (targetB == connection):
                counter.add(1)
            newEntry = (newTarget, ([], newDistance, newStatus))
            results.append(newEntry)
        # mark target as searched
        searchStatus = 'SEARCHED'
    
    results.append((target, (connections, distance, searchStatus)))
    
    return results

In [0]:
def reduceNode(data1, data2):
    connections1 = data1[0]
    connections2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    searchStatus1 = data1[2]
    searchStatus2 = data2[2]
    distance = 10000
    searchStatus = 'UNSEARCHED'
    connections = []
    if (len(connections) > 0):
        connections.extend(connections1)
    if (len(connections2) > 0):
        connections.extend(connections2)
    # Preserve minimum distance
    if (distance1 < distance):
        distance = distance1
    if (distance2 < distance):
        distance = distance2
    # Preserve the most advanced searchStatus
    if (searchStatus1 == 'NOT READY' and 
       (searchStatus2 == 'READY' or searchStatus2 == 'SEARCHED')):
           searchStatus = searchStatus2
    if (searchStatus1 == 'READY' and searchStatus2 == 'SEARCHED'):
           searchStatus = searchStatus2
    if (searchStatus2 == 'NOT READY' and 
       (searchStatus1 == 'READY' or searchStatus1 == 'SEARCHED')):
           searchStatus = searchStatus1
    if (searchStatus2 == 'READY' and searchStatus1 == 'SEARCHED'):
           searchStatus = searchStatus1
    
    return (connections, distance, searchStatus)

In [0]:
targetA = '0'
targetB = '3'
counter = sc.accumulator(0)

# Note that this is working for undirected graph, needs to be edited for directed graph

iteratingRDD = transformInput()
for iteration in range(0, 20):
    mapped = iteratingRDD.flatMap(mapNode)
    mapped.collect()
    if (counter.value > 0):
        print("Target B was found " + str(iteration+1) +  
        " levels from Target A and was connected to " +           
        str(counter.value) + " node at that level.")
        break
        
    # reduceByKey() is like combiner in mapreduce,
    # we are merging the values for a particular key,
    # then we are collecting the minimum distance
    iteratingRDD = mapped.reduceByKey(reduceNode)

Target B was found 2 levels from Target A and was connected to 1 node at that level.


In [0]:
input = sc.textFile("dbfs:/FileStore/tables/8lines.txt")
new = input.map(toNode)

In [0]:
def perform_bfs():
    # Note that this is working for undirected graph, needs to be edited for directed graph

    iteratingRDD = transformInput()
    for iteration in range(0, 20):
        mapped = iteratingRDD.flatMap(mapNode)
        mapped.collect()
        if (counter.value > 0):
            print(f"{targetA} to {targetB} distance is - {str(iteration+1)}")
            break

        # reduceByKey() is like combiner in mapreduce,
        # we are merging the values for a particular key,
        # then we are collecting the minimum distance
        iteratingRDD = mapped.reduceByKey(reduceNode)

In [0]:
checks = ['385','6','469']
counter = sc.accumulator(0)
def BFS(b):
    global targetA 
    targetA = '0'
    global targetB
    targetB = b
    global counter
    counter = sc.accumulator(0)
    perform_bfs()

In [0]:
#Now working correctly after adding global in BFS
counter = sc.accumulator(0)
for b in checks:
    BFS(b)

0 to 385 distance is - 2
0 to 6 distance is - 2
0 to 469 distance is - 1


In [0]:
# Not working properly
counter = sc.accumulator(0)
BFS('2')

Target B was found 3 levels from Target A and was connected to 1 node at that level.


In [0]:
# This is giving correct results
counter = sc.accumulator(0)
targetB = '385'
perform_bfs()

Target B was found 2 levels from Target A and was connected to 1 node at that level.


In [0]:
data = spark.createDataFrame(data=[('0','0')],schema=["Node","Distance from 0"])

In [0]:
targetA = "0"

def perform_return_bfs():
    # Note that this is working for undirected graph, needs to be edited for directed graph

    iteratingRDD = transformInput()
    for iteration in range(0, 20):
        mapped = iteratingRDD.flatMap(mapNode)
        mapped.collect()
        if (counter.value > 0):
            return str(iteration+1)
        # reduceByKey() is like combiner in mapreduce,
        # we are merging the values for a particular key,
        # then we are collecting the minimum distance
        iteratingRDD = mapped.reduceByKey(reduceNode)
    return '1000'
def BFS_return(b):
    global targetA 
    targetA = '0'
    global targetB
    targetB = b
    global counter
    counter = sc.accumulator(0)
    return perform_return_bfs()

In [0]:
for n in ['5','6']:
    new = spark.createDataFrame(data=[(n,BFS_return(n))],schema=["Node","Distance from 0"])
    data = data.union(new)

In [0]:
BFS_return('5')

Out[33]: '3'

In [0]:
perform_return_bfs()

Out[30]: '1'

In [0]:
data.show()

+----+---------------+
|Node|Distance from 0|
+----+---------------+
|   0|              0|
|   5|              3|
|   6|              2|
+----+---------------+



In [0]:
# trying to read edgelist
edgelist = sc.textFile("dbfs:/FileStore/tables/Graph.txt").map(lambda line: line.split("\t")).filter(lambda line: len(line) == 2)

#vertices = edgelist.select("src").union(edgelist.select("dst")).distinct().withColumnRenamed("src","id")
#vertices = dfedge.select("From").union(dfedge.select("To")).distinct().withColumnRenamed("From","id")

In [0]:
edges = spark.createDataFrame(data=[('0','0')],schema=["From","To"])

In [0]:
dfedge = edgelist.toDF(schema = edges.schema)

In [0]:
dfedge.show(1)